In [1]:
!pip install transformers torch gradio --quiet

import os
import random
from transformers import pipeline
import gradio as gr

In [2]:
sentiment_pipeline = pipeline("sentiment-analysis")
emoji_dict = {
    "POSITIVE": [
        "😃", "😊", "😁", "😎", "👍", "❤️", "🎉", "✨", "🌟", "🥳",
        "🤩", "😍", "👌", "💪", "🤗", "😄", "😺", "🌈", "🍀", "🎈",
        "🥰", "🍓", "🍰", "🕺", "💃", "🎶", "🎵", "☀️", "🌸", "🐱",
        "💫", "🌞", "🎁", "🌻", "🍩", "🍦", "🥇", "🏅", "🤸‍♂️", "🌹",
        "🐶", "🐰", "🦋", "🌼", "🏖️", "🍉", "🎂", "🍕", "🍓", "🚴‍♀️"
    ],
    "NEGATIVE": [
        "😢", "☹️", "😠", "💔", "😞", "😩", "😭", "😡", "👎", "😖",
        "😣", "😤", "🥀", "😿", "💀", "🥺", "😰", "😨", "👹", "😓",
        "😫", "🤯", "👎🏿", "🖤", "😟", "🧟", "👺", "☠️", "⚰️", "😬",
        "🕷️", "🦠", "😷", "🤒", "🤕", "😰", "😱", "😢", "🥀", "🩸",
        "👻", "⚡", "🔥", "🌪️", "🧨", "🥵", "🥶", "😥", "😓", "🧟‍♂️"
    ],
    "NEUTRAL": [
        "😐", "😶", "🤔", "🙂", "😌", "😑", "😬", "😴", "😕", "🙃",
        "😷", "🤨", "😳", "😶‍🌫️", "🤫", "🧐", "😒", "🧏", "🙄", "🛌",
        "🎭", "📊", "🎬", "🗿", "📚", "📰", "🤥", "😈", "🤡", "👻",
        "🕰️", "🗓️", "⌛", "⏳", "🔍", "🛋️", "📱", "💤", "🗺️", "🔄",
        "🔕", "🔔", "🚦", "🚧", "⚖️", "🕊️", "📌", "✏️", "✅", "❌"
    ]
}

keyword_emoji = {
    "happy":    ["😃", "😄", "😁", "🎈", "😆", "😂", "🥳", "😊", "🤗", "😀", "😺", "😸", "🏆", "🎉", "🎶"],
    "love":     ["❤️", "😍", "😘", "💖", "💕", "💘", "💝", "💞", "💟", "❣️", "💐", "🌹", "🌷", "💌", "🥰"],
    "angry":    ["😠", "😡", "😤", "👿", "🤬", "💢", "😾", "👺", "😾", "😒", "🔥", "😾", "💥", "🗯️", "😠"],
    "sad":      ["😢", "😥", "☹️", "😭", "😞", "😓", "😿", "🥺", "😔", "🙁", "🩸", "🧸", "🌧️", "☔", "🕯️"],
    "fun":      ["🎉", "😜", "😂", "🤪", "🎊", "🤣", "😹", "🥴", "😛", "🤩", "🎭", "🎪", "🎤", "🌟", "🎈"],
    "excited":  ["🤩", "😆", "🎊", "🙌", "🎉", "🥳", "🎈", "✨", "👑", "😃", "🕺", "💃", "🚀", "🌈", "🎶"],
    "surprised":["😲", "😳", "😯", "😮", "😱", "😧", "😵", "😮‍💨", "😨", "🤯", "😮‍💨", "🤭", "😶‍🌫️", "👀", "😵‍💫"],
    "bored":    ["😐", "😒", "😴", "😑", "🙄", "😪", "🥱", "😬", "😶", "😓", "🛌", "📉", "📚", "📺", "🕰️"],
    "scared":   ["😱", "😨", "😰", "👻", "😖", "😵‍💫", "😧", "😦", "😬", "🥶", "🦠", "🧟", "👹", "☠️", "🕷️"],
    "confused": ["😕", "🤷", "😵", "🧐", "🤔", "😐", "🙃", "😶", "😖", "😳", "🌀", "🤯", "😵‍💫", "🙄", "❓"]
}


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
def generate_emojis(text: str) -> str:
    # Sentiment prediction
    result = sentiment_pipeline(text)[0]
    label = result["label"]  # POSITIVE / NEGATIVE / NEUTRAL

    # Sample sentiment emojis
    sentiment_emojis = random.sample(emoji_dict[label], k=3)

    # Keyword-based emojis
    keywords = []
    for kw, em_list in keyword_emoji.items():
        if kw in text.lower():
            keywords.extend(em_list)
    keyword_emojis = random.sample(keywords, k=min(3, len(keywords))) if keywords else []

    # Combine and dedupe
    all_emojis = list(dict.fromkeys(sentiment_emojis + keyword_emojis))
    return " ".join(all_emojis)

In [6]:
def emoji_ui(text: str) -> str:
    return generate_emojis(text)

iface = gr.Interface(
    fn=emoji_ui,
    inputs=gr.Textbox(lines=2, placeholder="Type anything…"),
    outputs=gr.Textbox(label="Emoji Mood"),
    title="EmojiMood – Enhanced Emoji Generator",
    description="Enter a sentence or mood; EmojiMood returns matching emojis based on sentiment and keywords!"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a6e715bf3f108cd13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
